<a href="https://colab.research.google.com/github/Badar97/ProcessMining_Tirocinio/blob/main/Copy_of_NewBig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **BIG**

In [1]:
!pip install -U pm4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=a29f4ddaccb266d2885665148afb87e98131aeff6bc25c93aaa8c682594d253a
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
  Created wheel for stringdist: filename=StringDist-1.0.9-cp310-cp310-linux_x86_64.whl size=16966 sha256=330d3bb6e868a998f8140436cb4953a63282da263495d7d140e4cf88f37f8455
  Stored in directory: /root/.cache/pip/wheels/79/25/0a/4a0c2c7af31a8b473f6e10e30312396f31ae899722600d7b99
Successfully built intervaltree stringdist


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


##FIND CAUSAL RELATIONSHIP


This function finds the casual relationships from the petri-net of the process model. The inputs are the petri-net, the initial marking, and the final marking. The output is a list of python pairs where the second element directly follows the first element in the petri-net, which translates to a causal relationship.

In [3]:
#from pm4py.algo.discovery.footprints import algorithm as footprints_discovery
import pm4py
def findCausalRelationships(net):
    dict_succ=find_successors(net)
    result=[]
    for key,item in dict_succ.items():
        for s in item:
            result.append((key.label,s.label))
    return result


def find_successors_of_transition(net, transition):
    sources = {transition}
    targets = set()
    visited = set()
    while sources:
        source = sources.pop()
        if not (type(source) is pm4py.objects.petri_net.obj.PetriNet.Transition and source.label is not None):
            visited.add(source)
        for arc in source.out_arcs:
            if arc.target in visited:
                continue
            if type(arc.target) is pm4py.objects.petri_net.obj.PetriNet.Transition and arc.target.label is not None:
                targets.add(arc.target)
            else:
                sources.add(arc.target)
    return targets

def find_successors(net):
    return {transition: find_successors_of_transition(net, transition) for transition in net.transitions if transition.label is not None}

#Pick Aligned Trace

Questa funzione riceve in input la traccia, la rete di petri, il marking iniziale e finale. restituisce in output due liste, una contenente le tracce allineate al modello, l'altra con le tracce allineata rispetto al log.

In [5]:
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments


def pick_aligned_trace(trace, net, initial_marking, final_marking):
    aligned_traces = alignments.apply_trace(trace, net, initial_marking, final_marking)
    #print("alignment")
    #print(aligned_traces['alignment'])
    temp = []
    id = 0
    al = []
    temp1=[]
    id1=0
    fin = []
    for edge in aligned_traces['alignment']:
        id+=1
        temp.append((id,edge[1]))
    al.append(temp)
    for edge in aligned_traces['alignment']:
        id1+=1
        temp1.append((id1,edge[0]))
    fin.append(temp1)

    return al, fin


In [6]:
import pandas as pd
import pdb

def load_aligned_trace(trace, alignment_file):
    df=pd.read_csv(alignment_file, skiprows=1)
    boolean_find=df["Case IDs"].str.contains(trace.attributes["concept:name"])==True
    alignment=df[boolean_find]["Match"].values[0]
    moves=alignment.split("|")
    new_moves=list()
    for m in moves:
        kind=m.split("]")[0]
        act=m.split("]")[1]
        if kind=="[L/M":
            new_moves.append((act,act))
        elif kind=="[M-INVI":
            new_moves.append((">>",None))
        elif kind=="[M-REAL":
            new_moves.append((">>",act))
        elif kind=="[L":
            new_moves.append((act,">>"))
        else:
            print("unexpected move")
            print(m)

    temp = []
    id = 0
    al = []
    temp1=[]
    id1=0
    fin = []
    for edge in new_moves:
        id+=1
        temp.append((id,edge[1]))
    al.append(temp)
    for edge in new_moves:
        id1+=1
        temp1.append((id1,edge[0]))
    fin.append(temp1)

    return al, fin

#load_aligned_trace("requestforpayment154882",alignment_file)

##Check Trace Conformance

This function checks whether a given trace is conform to a petri-net through alignment based conformance checking. It takes in input the trace, the petri-net, the initial marking, and the final marking. It returns two lists: a list of sequences of deleted activities, and a list of sequences of inserted activities in the trace. Each element of the two lists is a list itself, this sublist contains pairs where the first element is the position of the deleted/inserted activity and the second is the activity label. In sequences of deleted activities the position value is always the same, while for inserted activities it's incremental.

In [7]:
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments


def checkTraceConformance(trace, net, initial_marking, final_marking):

    aligned_traces = alignments.apply_trace(trace, net, initial_marking, final_marking)
    D = []
    I = []
    id = 0
    temp_d = []
    temp_i = []
    prev_d = False
    curr_d = False
    prev_i = False
    curr_i = False
    del_count = 1
    for edge in aligned_traces['alignment']:
        pdb.set_trace()
        id+=1
        if edge[1] is None:
            id-=1
            continue
        if edge[0] == '>>':
            temp_d.append((id, edge[1]))
            curr_d = True
            id-=1
        if edge[1] == '>>':
            temp_i.append((id, edge[0]))
            curr_i = True

        if (prev_i and not curr_i):
            if len(temp_i) > 0:
                I.append(temp_i)
            temp_i = []
        prev_i = curr_i
        curr_i = False
        if (prev_d and not curr_d):
            if len(temp_d) > 0:
                D.append(temp_d)
            temp_d = []

        prev_d = curr_d
        curr_d = False
    if len(temp_i) > 0:
        I.append(temp_i)
    if len(temp_d) > 0:
        D.append(temp_d)
    return D, I


#MAPPING

La funzione mapping prende in input le due liste contenente le due tracce allineate. Crea quindi una lista di tuple di 3 elementi dove:
- il primo elemento della tupla corrisponde al nome dell'attività
- il secondo elemento della tupla corrisponde alla posizione dell'attività nella traccia presente nel log
- il terzo elemento della tupla corrisponde alla posizione dell'attività nella traccia allineata rispetto al modello; alle attività soggette a insertion (e quindi non presenti nel modello) viene assegnata la posizione a partire dalla lunghezza della traccia (quindi se ho una traccia di 10 elementi, la prima insertion avrà posizione 10+1, la seconda 10+2 ecc)

Il mapping permette di capire quali sono le attività inserite nel grafo, quindi la funzione restituisce anche una lista ins contenente le insertion.


In [8]:
def mapping(L1,L2): # crea il mapping, ritorna il map e la lista ins (lista degli inserimenti)
    map = [0]*len(L1)
    id1 = 0
    id2 = 0
    ins = []
    for i in range(len(L1)):
        e1=L1[i]
        e2=L2[i]
        if e1[1]==e2[1]:
            id1+=1
            id2+=1
            map[i] = (e1[1],id1, id2 )
        elif e1[1]=='>>': #insertion
            id1+=1
            map[i]= (e2[1],id1, 0)
        elif e2[1]=='>>': #deletion
            #pdb.set_trace()
            id2+=1
            map[i]=(e1[1],0,id2)


    for j in range(len(L1)):
        e1 = L1[j]
        e3 = map[j]
        if e1[1]=='>>':
            id2+=1
            map[j] = (e3[0], e3[1], id2)
            ins.append((e3[0], e3[1], id2))


    return map, ins




#Extract Instance Graph

This function extracts the instance graph of a trace. Every event of the trace is saved in a list V which represents the set of the nodes of the graph. An event is a pair of an ID (generated incrementally) and the activity label. The edges instead are saved as a pair of events in a list W. The algorithm is based on the definition 18 of the original paper.

In [9]:
def ExtractInstanceGraph(trace, cr):
    V = []
    W = []
    id = 1
    for event in trace:
      #V.append((id, event.get("concept:name")))
      V.append(event)
      id += 1
    # print("IG")
    for i in range(len(V)):
      for k in range(i+1,len(V)):
        e1 = V[i]
        e2 = V[k]
        if (e1[1],e2[1]) in cr:
          flag_e1=True
          for s in range(i+1, k):
            e3 = V[s]
            if (e1[1],e3[1]) in cr:
              flag_e1 = False
              break
          flag_e2=True
          for s in range(i+1, k):
            e3 = V[s]
            if (e3[1],e2[1]) in cr:
              flag_e2 = False
              break

          if flag_e1 or flag_e2:
            W.append((e1,e2))
    return V, W

#View Instance Graph

This function takes in input the list of nodes (events) and the list of edges and returns a GraphViz object. The other two inputs are view and the title of the graph. By default view is true so the function will show the graph in the output window.

In [10]:
from IPython import display
from graphviz import Digraph


def viewInstanceGraph(V, W, view=True, title="Instance Graph"):
  # Conversion to string indexes
  V2 = []
  W2 = []
  for node in V:
    V2.append((str(node[0]), "{0} = {1}".format(node[0],node[1])))
  for edge in W:
    W2.append(((str(edge[0][0]), "{0} = {1}".format(edge[0][0],edge[0][1])),(str(edge[1][0]), "{0} = {1}".format(edge[1][0],edge[1][1]))))

  print('V2: ', V2)
  print('W2: ', W2)

  dot = Digraph(comment=title, node_attr={'shape': 'circle'})
  for e in V2:
    dot.node(e[0], e[1])
  for w in W2:
    dot.edge(w[0][0], w[1][0])
  if view:
    display.display(dot)


  return dot

#Compliant trace

Prende in input la traccia ed elimina dalla stessa i move '>>'

In [11]:
def compliant_trace(trace):
  t = []
  id = 0
  for event in trace:
    if event[1] == '>>':
      continue
    else:
      id +=1
      t.append((id,event[1]))

  return t



#Deletion Repair

Questa funzione descrive il funzionamento della deletion repair.
Prende in input la lista dei nodi, la lista degli archi, il mapping e la lista delle deletion. restituisce in output i nuovi nodi e archi ottenuti a seguito della deletion repair.

In [12]:
#funzione per la deletion repair

def del_repair(V,W,map,deletion):
    Eremp = []
    Erems = []
    Pred = []
    Succ = []
    W1 = []
    V1 = []
    d = []
    W2 = []


    to_del = (deletion[2],deletion[0])

    for i in range(len(W)):
        e1 = W[i]
        e2 = e1[1]
        e3 = e1[0]
        if e2 == to_del:
            Eremp.append((e3,to_del))
        if e3 ==to_del:
            Erems.append((to_del,e2))

    for a in Eremp:   #crea liste Pred e Succ
        Pred.append(a[0])
    for b in Erems:
        Succ.append(b[1])

    for ep in Eremp:
        W.remove(ep)
    for es in Erems:
        W.remove(es)

    V.remove(to_del)

    for p in Pred:
        for s in Succ:
            W.append((p,s))


    return V,W




#Insertion Repair

Questa funzione descrive il procedimento dell'insertion repair.

In [13]:

#funzione di insertion repair


def ins_repair(V,W,map,insertion,V_n, ins_list, Vpos):
   Eremp = []
   Pred = []
   Succ = []
   pos_t = []
   W_num = []
   V1 = []
  # ins_num = []




   V.insert(insertion[1]-1,(insertion[2],insertion[0]))
   Vpos.insert(insertion[1]-1, (insertion[2], insertion[0]))
   pos_t.append(insertion[1])

   W_num = edge_number(W)
   V_num = node_number(V)
  # ins_num = ins_list_num(ins_list)

   #print('ins num: ',ins_num)

  # print('Vpos agg: ', Vpos)

   for p in pos_t: # numero dell'insertion
       #print('P=',p)
       #print('Len Vpos: ', len(Vpos))
       if p < len(Vpos):
         position = Vpos[p] #posizione in cui va inserito il nodo
        # print('Position = ', position)
         pos = position[0]
       else:   #inserimento a ultimo posto. la posizione di inserimento è maggiore o uguale della lunghezza di Vpos (che non considera nodi da cancellare)
         position = V[-1]
         pos = position[0]
         #print('ULtimo elemento ', Vpos[-2])
         #print(pos)

       print('pos: ', pos)
       p_pred = Vpos[p-2] #in Vpos il nodo da inserire viene messo in posizione p-1 perchè il vettore parte da 0, quindi il precedente lo trovo come p-2
       pos_pred = p_pred[0]
      # print('P pred: ', p_pred)
       #linee 6-12 pseudocodice
       if is_path(pos_pred,pos,W_num,V):  # se c'è un cammino tra p-1 e p
         print(is_path(pos_pred,pos,W_num,V))
         for i in range(len(W)):
           arc = W[i]
           a0 = arc[0]
           a1 = arc[1]
           if pos == a1[0] and (a0,a1) not in Eremp:
             # se esiste un arco nel grafo che entra in posizione p e non è ancora stato inserito in Eremp
             # si trovano gli archi entranti (e quindi i nodi Pred) nel nodo in posizione in cui va fatto l'inserimento
             Eremp.append((a0,a1))
             Pred.append(a0)
         for n in Pred:        #linee 9-10 pseudocodice, si controllano eventuali parallelismi non considerati nel ciclo precedente
           for k in range(len(W)):
             e = W[k]
             e0 = e[0]
             e1 = e[1]
             if e0 == n and (e0,e1) not in Eremp:
               Eremp.append((e0,e1))
       else: #linee 14-15 pseudocodice, l'insertion avviene all'interno di un parallelismo
         for m in range(len(W)):
           edge = W[m]
           edge0 = edge[0]
           edge1 = edge[1]
           if (pos_pred) == edge0[0] and (edge0,edge1) not in Eremp:
             Eremp.append((edge0,edge1))
             Pred.append(edge0)
           elif (pos_pred) == edge1[0] and (pos_pred) == V_n[-1]:  #insertion all'ultimo nodo del grafo
             Pred.append(edge1)




   #linea 17 pseudocodice
   for erem in range(len(Eremp)):
     suc = Eremp[erem]
     suc1 = suc[1]
     if suc1 not in Succ:
       Succ.append(suc1)

   #print('Pred = ', Pred)
   #print('Succ = ', Succ)
   #print('Eremp = ', Eremp)

   #linea 18 pseudocodice
   for el in Eremp:
     if el in W:
       W.remove(el)


   for i in Pred:
     if (i,(insertion[2],insertion[0])) not in W:
       W.append((i,(insertion[2],insertion[0])))

   for s in Succ:
     if ((insertion[2],insertion[0]),s) not in W:
       W.append(((insertion[2],insertion[0]),s))


   W_num = edge_number(W)
   V_num = node_number(V)

  # print('V: ', V)
   #print('VPOS Finale: ', Vpos)

   #print('++++++++++++++')


   return V,W


#Edge number

Questa funzione prende in input la lista degli archi, dove ogni arco è rappresentato come coppia ((1,A),(2,B)) e restituisce la lista di archi rappresentati solo come coppie numeriche (1,2). Questa funzione è utilizzata all'interno dell'insertion repair.

In [14]:
#crea una lista dove gli archi sono riportati solo come coppia di numeri

def edge_number(W):

  W_number = []

  for i in range(len(W)):
    arc = W[i]
    a0 = arc[0]
    a1 = arc[1]
    W_number.append((a0[0],a1[0]))

  return W_number

#Node number

Questa funzione prende in input una lista di coppie (id, label) corrispondenti ad ogni nodo del grafo. Restituisce una lista contenente solo il numero di ogni nodo.

In [15]:
#funzione che, data in input una lista di coppie (numero, label) corrispendenti ad ogni nodo del grafo,
# restituisce una lista con solo i numeri di ogni nodo

def node_number(V):

  V_number = []
  for i in range(len(V)):
    nod = V[i]
    V_number.append(nod[0])

  return V_number

#Is Path

Funzione booleana per controllare se, dati due nodi a e b in input, esista un cammino che li collega. Viene utilizzata durante l'insertion repair per gestire i parallelismi.

In [16]:
#funzione booleana per verificare se tra due nodi dati in input esiste un cammino che li collega

def is_path(a,b,W,V):
  flag = False
  if (a,b) in W:
    flag = True
    return flag
  else:
    for c in range(len(V)):
      e = V[c]
      if (a,e[0]) in W:
        flag = is_path(e[0],b,W,V)
      else:
        continue

  return flag


# Aggiorna label

Questa funzione, invocata al termine del repairing, aggiorna le label dei nodi in base a quanto contenuto nel mapping.

In [17]:
#aggiorna le label dei nodi in base al mapping

def aggiorna_label(W,map,V):

  W1 = []
  V1 = []


  for i in range(len(W)):
    arc = W[i]
    a0 = arc[0]
    a1 = arc[1]
    for j in range(len(map)):
      e = map[j]
      if a0 == (e[2],e[0]):
        for k in range(len(map)):
          f = map[k]
          if a1 == (f[2],f[0]):
            W1.append(((e[1],e[0]),(f[1],f[0])))


  for i1 in range(len(V)):
    node = V[i1]
    for j1 in range(len(map)):
      e = map[j1]
      if node == (e[2],e[0]):
        V1.append((e[1],e[0]))


  return W1, V1



##SAVE FILE


In [18]:
def saveGFile(V, W, path,time, sort_labels):
  with open(path, 'w') as f:
    f.write("# Execution Time: {0:.3f} s\n".format(time))
    #f.write("# Deleted Activities: {0}\n".format(D))
    #f.write("# Inserted Activities: {0}\n".format(I))
    for n in V:
      f.write("v {0} {1}\n".format(n[0], n[1]))
    f.write("\n")
    if (sort_labels):
      W.sort()
    for e in W:
      f.write("e {0} {1} {2}__{3}\n".format(e[0][0], e[1][0], e[0][1], e[1][1]))


In [19]:
def saveGfinal(V, W, path, sort_labels):
  with open(path, 'a') as f:
    f.write("XP \n")
    for n in V:
      f.write("v {0} {1}\n".format(n[0], n[1]))
    if (sort_labels):
      W.sort()
    for e in W:
      f.write("e {0} {1} {2}__{3}\n".format(e[0][0], e[1][0], e[0][1], e[1][1]))
    f.write("\n")
    f.close()


In [20]:
def saveCSV(path, aligned, model_moves, num):
  with open(path, 'a') as f:
    writer=csv.writer(f,delimiter=";")
    writer.writerow([num, aligned, model_moves ])
    f.close()

In [21]:
def make_visible(data, admissible_tau_names=None):
    invisible_names = ['tau split', 'tau join', 'tau from tree','tau start']
    if admissible_tau_names is None:
        admissible_tau_names = []
    def replacement(match):
        orig_ident = match.group(2)
        orig_name = match.group(4)
        is_invisible = False
        if 'tau' in orig_name:
            if orig_name in invisible_names:
                is_invisible = True
            elif orig_name not in admissible_tau_names:
                raise Exception(f'Encountered "tau" in transition name "{orig_name}"; add to admissible_tau_names or invisible_names to suppress this error message')
        ident = orig_ident
        name = f'{orig_ident} {orig_name}'.replace(' ', '-') if is_invisible else orig_name# matches regular expression '^n[0-9]+-tau-' or contains '-tau-'
        activity = name
        return match.group(1) + ident + match.group(3) + name + match.group(5)# + activity + match.group(7)
    return re.compile('(<transition id=\")([^\"]*)(\">.*?<name><text>)(.*?)(</text></name>)').sub(replacement, data)

def make_invisible(net):
    invisible_names = ['tau split', 'tau join', 'tau from tree','tau start']
    for t in net.transitions:
        if t.label!="" and t.label!=None and "tau" in t.label:
            if t.label in invisible_names:
                t.label=None
            else:
                raise Exception(f'Encountered "tau" in transition name "{orig_name}"; add to admissible_tau_names or invisible_names to suppress this error message')
    return net


##MAIN

In [22]:
from pm4py.streaming.importer.xes import importer as xes_importer
import time
from pm4py.objects.petri_net.importer import importer as pnml_importer
from pm4py.objects.petri_net.exporter import exporter as pnml_exporter

import graphviz
import csv
import pydot
import pdb
import re
import networkx
import os
import pdb


from pm4py.visualization.petri_net import visualizer as pn_visualizer

alignment_file="C:/Users/lgenga/Documents/TUEAttivita/Ancona/Andrea/revised_jiis/bpi12_w/Conformance/alignment.csv"
log_file = '/content/drive/MyDrive/Tirocinio/dataset_new/BPI13O.xes'
net_file = '/content/drive/MyDrive/Tirocinio/dataset_new/dgcnn_log_sample_net_startend.pnml'
log_file1 = '/content/drive/MyDrive/DWH/BIG Datasets/bpi2012decompositionExpr/bpi2012decompositionExpr.xes'
net_file1 = '/content/drive/MyDrive/DWH/BIG Datasets/bpi2012decompositionExpr/bpi2012decompositionExpr_petriNet.pnml'
log_file2 = '/content/drive/MyDrive/DWH/BIG Datasets/BPI2017Denied/BPI2017Denied.xes'
net_file2 = '/content/drive/MyDrive/DWH/BIG Datasets/BPI2017Denied/BPI2017Denied_petriNet.pnml'
log_file3 = '/content/drive/MyDrive/DWH/BIG Datasets/Hospital_dcc/Hospital_dcc.xes'
net_file3 = '/content/drive/MyDrive/DWH/BIG Datasets/Hospital_dcc/Hospital_dcc_petriNet.pnml'
log_file4 = '/content/drive/MyDrive/DWH/BIG Datasets/testBank2000SCCUpdatedCopia/testBank2000SCCUpdatedCopia.xes'
net_file4 = '/content/drive/MyDrive/DWH/BIG Datasets/testBank2000SCCUpdatedCopia/testBank2000SCCUpdatedCopia_petriNet.pnml'


def BIG(net_path, log_path, tr_start=0, tr_end=None, view=False, sort_labels=False):
    splits = log_path.split('/')
    name = splits[-1].split(".")[0]

    streaming_ev_object = xes_importer.apply(log_path, variant=xes_importer.Variants.XES_TRACE_STREAM) #file xes

    #pdb.set_trace()
    #Aggiunta Laura 19/08
    net_modified_file=net_path.replace(".pnml","_visible.pnml")
#     with open(net_path) as infile:
#         with open(net_modified_file, 'w') as outfile:
#             outfile.write(make_visible(infile.read()))
    net, initial_marking, final_marking = pnml_importer.apply(net_path)
    net=make_invisible(net)
    pnml_exporter.apply(net,initial_marking,net_modified_file,final_marking)

    gviz = pn_visualizer.apply(net, initial_marking, final_marking)
    #display.display(gviz)
    gviz.render(filename="petri")

    #start_time_total = time.time()
    cr = findCausalRelationships(net)
    if view:
        print(cr)

    n = 0

    Aligned=[]
    L = []
    Align = []
    L1 = []
    A = []
    A1=[]
    map = []
    compliant = []
    ins =  []

    start_time_total = time.time()


  #with open("{0}_instance_graphs.csv".format(name), 'w') as f:
   # writer=csv.writer(f,delimiter=";")
    #writer.writerow(['n', 'aligned to model', 'with invisible moves'])
    #f.close()
#     check_list=list()
#     for trace in streaming_ev_object:
#         #print(trace)
#         if trace.attributes.get('concept:name')=="173694":
#             check_list.append(trace)
#             print("added")

    #:
    for trace in streaming_ev_object:
        n += 1
        print("Doing trace "+str(n))
        #pos trace
        Aligned, A = pick_aligned_trace(trace, net, initial_marking, final_marking)
        #Aligned, A = load_aligned_trace(trace,alignment_file)

        Align = Aligned[0]
        A1 = A[0]
        if view:
            print('Aligned to model')
            print(Align)
            print('with invisible moves')
            print(A1)
        map, ins = mapping(Align,A1)



        #compliant mi serve per generare l'ig in base al modello (rimuove dalla traccia allineata i move)
        compliant = compliant_trace(Align)
        effettiva = compliant_trace(A1)

        if view:
            print(compliant)
            print('Effettiva: ', effettiva)

            print("map: ", map)
            print("ins: ", ins)

        d = []

        trace_start_time = time.time()
        num = trace.attributes.get('concept:name')
        id = trace.attributes.get('variant-index')

        #estrazione dell' IG su cui poi devo fare riparazione
        V, W = ExtractInstanceGraph(compliant,cr)
        #viewInstanceGraph(V,W)
        if view:
            print('V')
            print(V)
            print('W')
            print(W)


        #ottengo liste di nodi e archi contenenti solo i valori numerici dei nodi
        V_n = node_number(V)
        W_n = edge_number(W)


        for element in map:     #crea lista dei nodi da cancellare
            if element[1]== 0:
                d.append(element)

        #Vpos lista dei nodi utilizzata per repair. inizialmente si rimuovono da essa i nodi relativi a deletion.
        # In seguito viene passata in input alla funzione di insertion repair e ogni attività viene inserita all'interno di Vpos
        # nella posizione di inserimento. così facendo vpos sarà sempre aggiornata ad ogni inserimento.
        #al termine delle insertion avrò la mia lista dei nodi aggiornata.


        Vpos = []
        for node in V:
            Vpos.append(node)


        for el in map:
            if el[1]== 0:
                Vpos.remove((el[2], el[0]))

        if view:
            print('Vpos = ', Vpos)

            print('INSERTION REPAIR')

        for insertion in ins:
            V,W = ins_repair(V,W,map,insertion,V_n,ins,Vpos)

        if view:
            print('W repaired: ', W)

        # if n==1500:
         # viewInstanceGraph(V,W)


        if view:
            print('DELETION REPAIR')
            print(d)

        for deletion in d:

            V,W = del_repair(V,W,map,deletion)


        #if n==1500:
         #viewInstanceGraph(V,W)




        #aggiorna le label dei nodi in base a quanto contenuto nel mapping

        W_new, V_new = aggiorna_label(W,map,V)

        #riordina le liste di nodi e archi in base agli id
        V_new.sort()
        W_new.sort()

        if view:
            print('V finale: ',V_new)
            print('W finale: ',W_new)

        #if n==7455:
         #dot = viewInstanceGraph(V_new,W_new)
         #dot.save()

        #dot = viewInstanceGraph(V,W)
        #dot.save()



        W_new_digraph = networkx.transitive_reduction(networkx.DiGraph([(a[0], b[0]) for (a, b) in W_new]))
        V_new_dict = {number: (number, label) for (number, label) in V_new}
        W_new_reduced = [(V_new_dict[a], V_new_dict[b]) for (a, b) in sorted(W_new_digraph.edges())]
        W_new = W_new_reduced


        elapsed = time.time() - start_time_total



        saveGFile(V_new, W_new, os.path.dirname(log_file)+"/"+os.path.basename(log_file).replace(".xes","log_ig"), time.time()-trace_start_time, sort_labels)
        #saveGfinal(V_new, W_new, "{0}_instance_graphs.g".format(name), sort_labels)
        saveGfinal(V_new, W_new, "/content/drive/MyDrive/Tirocinio/dataset_new/instance_graphs.g", sort_labels)



        if view:
            dot = viewInstanceGraph(V_new,W_new)


        print('------------------------------------------------------------------')




        print('Time: ', elapsed)


splits = log_file.split('/')
name = splits[-1].split(".")[0]
if os.path.exists("{0}_instance_graphs.g".format(name)):
    os.remove("{0}_instance_graphs.g".format(name))
    print("Previous IG file removed")
BIG(net_file,log_file)

/usr/local/lib/python3.10/dist-packages/pm4py/util/dt_parsing/parser.py:77: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


Doing trace 1
------------------------------------------------------------------
Time:  0.03306150436401367
Doing trace 2
------------------------------------------------------------------
Time:  0.04704713821411133
Doing trace 3
------------------------------------------------------------------
Time:  0.06132626533508301
Doing trace 4
------------------------------------------------------------------
Time:  0.07696318626403809
Doing trace 5
------------------------------------------------------------------
Time:  0.09361958503723145
Doing trace 6
------------------------------------------------------------------
Time:  0.10965371131896973
Doing trace 7
------------------------------------------------------------------
Time:  0.12251853942871094
Doing trace 8
------------------------------------------------------------------
Time:  0.13557863235473633
Doing trace 9
------------------------------------------------------------------
Time:  0.16693520545959473
Doing trace 10
-------------

In [ ]:
import pm4py
import pdb
from pm4py.objects.petri_net.importer import importer as pnml_importer
from pm4py.streaming.importer.xes import importer as xes_importer


log_file = 'C:/Users/lgenga/Documents/TUEAttivita/Ancona/Andrea/bpi2012/BPI2012_SE.xes'
net_file = 'G:/My Drive/EcondingProject/ICPM2022/PrepaidTravelCost/petriNet_03.pnml'
net, initial_marking, final_marking = pnml_importer.apply(net_file)
streaming_ev_object = xes_importer.apply(log_file, variant=xes_importer.Variants.XES_TRACE_STREAM)
n=0
for trace in streaming_ev_object:
    n=n+1
print(n)
    #streaming_ev_object = pm4py.read_xes(log_file)
#aligned_traces = pm4py.conformance_diagnostics_alignments(streaming_ev_object, net, initial_marking, final_marking)

In [ ]:
import networkx
g = networkx.DiGraph([(1, 2), (2, 3), (3, 4), (1, 4), (1, 4), (4, 5), (5, 6), (6, 7), (7, 8)])
t = networkx.transitive_reduction(g)

In [ ]:
t.edges

In [ ]:
for r in cr:
    print(r)

In [ ]:
from pm4py.objects.petri_net.importer import importer as pnml_importer
import pdb

net_file = 'C:/Users/lgenga/Documents/TUEAttivita/Ancona/Andrea/bpi2012/andrea_bpi12full_petriNet_20.pnml'
net, initial_marking, final_marking = pnml_importer.apply(net_file)
net=make_invisible(net)
fp_net = footprints_discovery.apply(net, initial_marking, final_marking)
list(fp_net.get('sequence'))

In [ ]:
def find_succ(t, succ_list,checked,i):
    i=i+1
    print("call "+str(i))
#     print("checking node "+str(t))
    for arc in t.out_arcs:
        for arc2 in arc.target.out_arcs:
            if arc2.target not in checked:
                succ_list.append(arc2.target)
    #print("updated succ list "+str(succ_list))
    inv_list=[]
    for s in succ_list:
        if s.label==None:
            inv_list.append(s)
    checked.append(t)
    #print("update checked "+str(checked))
    succ_list = [s for s in succ_list if s not in inv_list]
    for s in inv_list:
        #pdb.set_trace()
        new_succ=find_succ(s,succ_list,checked,i)
        for el in new_succ:
            if el not in succ_list and el.label!=None:
                succ_list.append(el)
    print("succ_list call "+str(i)+" "+str(succ_list))
    return succ_list

succ_dic=dict()
for t in net.transitions:
    candidates=[]
    succ_list=find_succ(t,[],[],0)
    succ_dic[t.name]=succ_list
    break

In [ ]:
log_file = 'C:/Users/lgenga/Documents/TUEAttivita/Ancona/Andrea/bpi2012/BPI2012_SE.xes'
net_file = 'C:/Users/lgenga/Documents/TUEAttivita/Ancona/Andrea/bpi2012/andrea_bpi12full_petriNet_20.pnml'
splits = log_file.split('/')
name = splits[-1].split(".")[0]

streaming_ev_object = xes_importer.apply(log_file, variant=xes_importer.Variants.XES_TRACE_STREAM) #file xes

net_modified_file=net_file.replace(".pnml","_visible.pnml")
net, initial_marking, final_marking = pnml_importer.apply(net_file)
net=make_invisible(net)

n=0
for trace in streaming_ev_object:
    n=n+1
    if n==3:
        aligned_traces = alignments.apply_trace(trace, net, initial_marking, final_marking)
        print(aligned_traces)
